<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/crewai_multiagent_Gitex_level_2_29th_nov_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this workshop, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [1]:
#install libraries
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai import Agent, Task, Crew, Process

# --- Agents ---
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,

)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,   # manager must be allowed to delegate
    verbose=True,

)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,

)

def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    # --- Tasks ---
    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    # --- Crew (hierarchical: strategist manages delegation/routing) ---
    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        #process=Process.hierarchical,
        #manager_agent=chief_marketing_strategist,
        verbose=True
    )

    return crew.kickoff()


In [ ]:
brand     = "SaaS analytics platform"
audience  = "Mid-market product managers in EMEA"
objective = "Drive free trials"

result = run_marketing_crew(brand, audience, objective)
print("\n=== OUTPUT ===\n")
print(result)


 [DEBUG]: == Working Agent: Lead Market Analyst
 [INFO]: == Starting Task: Conduct a concise market analysis for: SaaS analytics platform | Audience: Mid-market product managers in EMEA | Objective: Drive free trials. Cover: (1) ICP & segments, (2) JTBD/pain points & objections, (3) competitive landscape & whitespace, (4) demand signals & seasonality, (5) channel dynamics (search/social/email/partners/events), (6) keyword themes & content gaps, (7) risks/assumptions.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

1. ICP & Segments:
- Ideal Customer Profile (ICP) for the SaaS analytics platform includes mid-market product managers in EMEA.
- Segments within this audience could be based on industry verticals, company size, or specific pain points related to analytics needs.

2. JTBD/Pain Points & Objections:
- Jobs-to-be-Done (JTBD) for mid-market product managers in EMEA could include improving product performance, optimizing pricing strategie

# Deploy this crew in gradio

In [6]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.20.0 which is incompatible.


In [7]:
import gradio as gr
from crewai import Agent, Task, Crew, Process

# ---------- Agents ----------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for {product_brand}",
    backstory=(
        "A senior analyst skilled in competitor intelligence, audience segmentation, "
        "channel dynamics, and market sizing, with a bias for actionable insights."
    ),
    allow_delegation=False,
    verbose=True
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy for {product_brand}",
    backstory=(
        "A veteran strategist who crafts positioning, messaging pillars, channel mix, "
        "and KPI frameworks—aligning cross-functional stakeholders and timelines."
    ),
    # Manager/orchestrator: can hand off sub-tasks or ask questions to teammates
    allow_delegation=True,
    verbose=True
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling creative concepts and a content calendar",
    backstory=(
        "A concept-to-copy creative who converts strategy into campaign ideas, ad copy, "
        "social posts, and SEO-ready long-form content."
    ),
    allow_delegation=False,
    verbose=True
)

# ---------- Crew runner ----------
def run_crewai_marketing_strategy(product_brand: str, target_audience: str, objective: str):
    # Compose a shared topic string for clarity across tasks
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    # 1) Market Analysis
    market_analysis_task = Task(
        description=(
            "Conduct a concise but thorough market analysis for the topic: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section above, "
            "ending with a 5–8 point summary of the most actionable insights."
        ),
        agent=lead_market_analyst,
        input_variables={"topic": topic}
    )

    # 2) Strategy Formulation (manager)
    strategy_task = Task(
        description=(
            "Using the Market Analysis brief, craft a go-to-market strategy for {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output=(
            "A strategy document with clear sections as listed, plus a one-page executive summary."
        ),
        agent=chief_marketing_strategist,
        input_variables={"topic": topic},
        context=[market_analysis_task]
    )

    # 3) Creative & Content Plan
    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search, paid social), "
            "(c) a 4-week content calendar (blog/LI/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, and intended KPIs, and (d) landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output=(
            "Campaign concepts + copy, a tabular content calendar, and a structured LP outline."
        ),
        agent=creative_content_creator,
        input_variables={"topic": topic},
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task]

    )

    result = crew.kickoff()
    return result

# ---------- Gradio app ----------
def generate_strategy(product_brand, target_audience, objective):
    return run_crewai_marketing_strategy(product_brand, target_audience, objective)

iface = gr.Interface(
    fn=generate_strategy,
    inputs=[
        gr.Textbox(lines=1, label="Product / Brand", placeholder="e.g., SaaS analytics platform"),
        gr.Textbox(lines=1, label="Target Audience", placeholder="e.g., mid-market product managers in EMEA"),
        gr.Textbox(lines=1, label="Primary Objective", placeholder="e.g., drive free trials / demo requests")
    ],
    outputs=gr.Textbox(lines=28, label="Marketing Strategy & Content Plan"),
    title="AI Agent Post to Prompt",
    description="Provide brand, audience, and objective. The crew analyzes the market, builds a GTM strategy, and outputs creative + a content calendar."
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4806cf6eef7c9ed68e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##Adding more advanced feature
Added one more layer for adding more options for social media

In [ ]:
from crewai import Agent, Task, Crew
import gradio as gr
import re
from datetime import datetime
from pathlib import Path

# ----------------------------
# Your existing agents
# ----------------------------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,
    verbose=True,
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,
)

# Optional: a focused social copywriter (only used if toggled)
social_copywriter = Agent(
    role="Social Copywriter",
    goal="Turn strategy into platform-appropriate copy with strong hooks and clear CTAs.",
    backstory=("Skilled at LinkedIn thought-leadership, X brevity, Instagram captions, "
               "and long-form posts that convert."),
    allow_delegation=False,
    verbose=False,
)

# ----------------------------
# Core crew (kept same logic)
# ----------------------------
def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        verbose=True
    )
    return crew.kickoff()  # returns the full text output

# ----------------------------
# Helpers (small + simple)
# ----------------------------
def _first_n_points(text: str, n: int = 5):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    bullets = []
    for l in lines:
        if l.startswith(("-", "*", "•")) or re.match(r"^\d+[\.\)]", l) or len(l) > 50:
            bullets.append(l.lstrip("-*• ").strip())
        if len(bullets) >= n: break
    if not bullets:
        parts = [p.strip() for p in text.split("\n\n") if p.strip()]
        bullets = parts[:n]
    return bullets[:n]

def _hashtags(csv_tags: str) -> str:
    tags = [t.strip().replace("#", "") for t in (csv_tags or "").split(",") if t.strip()]
    return "" if not tags else " " + " ".join(f"#{t}" for t in tags)

def _truncate_chars(s: str, max_chars: int) -> str:
    return s if len(s) <= max_chars else s[:max_chars - 1] + "…"

# ----------------------------
# Templates (no extra LLM call)
# ----------------------------
def tpl_linkedin(strategy, brand, audience, objective, hashtags, max_words=180):
    hook = f"{brand}: a sharper path to {objective} for {audience}."
    pts = "\n".join([f"- {p}" for p in _first_n_points(strategy, 5)])
    body = f"""**{hook}**

**Key insights**
{pts}

**Next 90 days**
- Prove the positioning with fast tests
- Double down on channels with strongest signals
- Track leading KPIs weekly

CTA: Comment “PLAYBOOK” if you want the GTM outline.{_hashtags(hashtags)}
"""
    words = body.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else body

def tpl_tweet(strategy, brand, audience, objective, hashtags, max_chars=270):
    points = _first_n_points(strategy, 3)
    msg = f"{brand} → {objective} for {audience}: " + " | ".join(_truncate_chars(p, 80) for p in points)
    return _truncate_chars(msg + _hashtags(hashtags), max_chars)

def tpl_article(strategy, brand, audience, objective, hashtags, max_words=800):
    intro = (f"{brand} is targeting {audience} to achieve {objective}. "
             "Here’s the distilled market analysis, GTM strategy, and a 90-day plan.")
    text = f"""# {brand}: GTM Playbook for {audience}

## Why this matters
{intro}

## Strategy in brief
- Positioning & messaging pillars
- Priority segments & channel mix
- Offers/CTAs, KPI tree
- 90-day roadmap & budget split

## Research & Strategy Notes
{strategy}

## What to do next
1) Validate 1–2 offers with tight ICP cohorts
2) Launch a 2-channel test with weekly KPI reviews
3) Scale what converts, archive what doesn’t

*Updated: {datetime.utcnow().strftime("%Y-%m-%d")}*{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

def tpl_instagram(strategy, brand, audience, objective, hashtags, max_chars=2200):
    pts = _first_n_points(strategy, 3)
    caption = (f"{brand} | {objective} for {audience}\n"
               + "\n".join([f"• {p}" for p in pts])
               + "\n\nCTA: Save & share with your team." + _hashtags(hashtags))
    return _truncate_chars(caption, max_chars)

def tpl_facebook(strategy, brand, audience, objective, hashtags, max_chars=3000):
    pts = _first_n_points(strategy, 4)
    post = (f"{brand} — how we’ll reach {audience} and drive {objective}:\n"
            + "\n".join([f"- {p}" for p in pts])
            + "\n\nComment if you want the 90-day playbook." + _hashtags(hashtags))
    return _truncate_chars(post, max_chars)

def tpl_youtube_desc(strategy, brand, audience, objective, hashtags, max_chars=5000):
    pts = _first_n_points(strategy, 5)
    desc = (f"{brand} GTM for {audience} | Objective: {objective}\n\nKey Points:\n"
            + "\n".join([f"- {p}" for p in pts])
            + "\n\nChapters:\n00:00 Intro\n00:30 Market Signals\n02:00 Strategy\n04:00 Next Steps\n"
            + "\nCTA: Subscribe for weekly GTM breakdowns." + _hashtags(hashtags))
    return _truncate_chars(desc, max_chars)

def tpl_threads(strategy, brand, audience, objective, hashtags, max_chars=500):
    pts = _first_n_points(strategy, 2)
    post = f"{brand} → {objective} for {audience}\n" + "\n".join([f"• {p}" for p in pts]) + _hashtags(hashtags)
    return _truncate_chars(post, max_chars)

def tpl_medium(strategy, brand, audience, objective, hashtags, max_words=1000):
    # Medium: same as article but lighter header
    text = f"""# {brand} GTM: From Signals to Scale

**Audience:** {audience} | **Objective:** {objective}

**In this post:** positioning, channel mix, KPIs, and a practical 90-day plan.

## Insights & Plan
{strategy}

## Close
If this resonates, highlight what you’d test first and why.{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

# ----------------------------
# Optional LLM copywriter (reuses Crew/Agent)
# ----------------------------
def llm_copywriter(strategy_text, brand, audience, objective, tone, platform,
                   hashtags, li_words, tweet_chars, article_words):
    limits = {
        "LinkedIn": f"≈ {li_words} words",
        "X (Twitter)": f"≤ {tweet_chars} chars",
        "Article": f"≈ {article_words} words",
        "Instagram": "≤ 2200 chars",
        "Facebook": "≤ 3000 chars",
        "YouTube Description": "≤ 5000 chars",
        "Threads": "≤ 500 chars",
        "Medium Article": f"≈ {article_words} words",
    }
    req = f"""Create a {platform} post from the GTM strategy.

Brand: {brand}
Audience: {audience}
Objective: {objective}
Tone: {tone}
Hashtags: {hashtags or '(none)'}
Length limit: {limits.get(platform, 'keep concise')}

Requirements:
- Strong first-line hook
- 1–3 concrete insights from the strategy (no clichés)
- Clear CTA
- Respect platform style and length

--- STRATEGY ---
{strategy_text}
"""
    task = Task(description=req, expected_output=f"{platform} copy ready to publish.", agent=social_copywriter)
    crew = Crew(agents=[social_copywriter], tasks=[task], verbose=False)
    return crew.kickoff()

# ----------------------------
# Gradio glue (simple dropdown)
# ----------------------------
PLATFORMS = [
    "LinkedIn",
    "X (Twitter)",
    "Article",
    "Instagram",
    "Facebook",
    "YouTube Description",
    "Threads",
    "Medium Article",
]

def generate(product_brand, target_audience, objective,
             platform, tone, hashtags, use_llm,
             li_max_words, tweet_max_chars, article_max_words):

    if not product_brand or not target_audience or not objective:
        return "Please fill Brand, Audience, and Objective.", "", None

    # 1) Run your main Crew once
    strategy = run_marketing_crew(product_brand, target_audience, objective)

    # 2) Pick template or LLM route
    if use_llm:
        social = llm_copywriter(strategy, product_brand, target_audience, objective,
                                tone, platform, hashtags,
                                li_max_words, tweet_max_chars, article_max_words)
    else:
        if platform == "LinkedIn":
            social = tpl_linkedin(strategy, product_brand, target_audience, objective, hashtags, li_max_words)
            fname = f"linkedin_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "X (Twitter)":
            social = tpl_tweet(strategy, product_brand, target_audience, objective, hashtags, tweet_max_chars)
            fname = f"tweet_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Article":
            social = tpl_article(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"article_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "Instagram":
            social = tpl_instagram(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"instagram_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Facebook":
            social = tpl_facebook(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"facebook_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "YouTube Description":
            social = tpl_youtube_desc(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"youtube_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Threads":
            social = tpl_threads(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"threads_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        else:  # Medium Article
            social = tpl_medium(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"medium_{re.sub(r'\\W+','_',product_brand.lower())}.md"

        # Save to file in Colab for download
        out_path = Path(fname).resolve()
        out_path.write_text(social, encoding="utf-8")
        return strategy, social, str(out_path)

    # If LLM path (single name based on platform)
    map_name = {
        "LinkedIn": "linkedin",
        "X (Twitter)": "tweet",
        "Article": "article",
        "Instagram": "instagram",
        "Facebook": "facebook",
        "YouTube Description": "youtube",
        "Threads": "threads",
        "Medium Article": "medium",
    }
    fname = f"{map_name.get(platform,'post')}_{re.sub(r'\\W+','_',product_brand.lower())}.md"
    out_path = Path(fname).resolve()
    out_path.write_text(social, encoding="utf-8")
    return strategy, social, str(out_path)

# ----------------------------
# Build the simple UI (good for Colab)
# ----------------------------
with gr.Blocks(title="Marketing Crew → Social Content") as demo:
    gr.Markdown("## Marketing Strategy → Social Content")

    with gr.Row():
        brand = gr.Textbox(label="Product/Brand", placeholder="e.g., DDS AI Residency")
        audience = gr.Textbox(label="Target Audience", placeholder="e.g., Data science beginners in MENA")
        objective = gr.Textbox(label="Objective", placeholder="e.g., Drive applications for Cohort 8")

    with gr.Row():
        platform = gr.Dropdown(choices=PLATFORMS, value="LinkedIn", label="Platform")
        tone = gr.Dropdown(choices=["Professional", "Friendly", "Bold", "Educational", "Conversational"],
                           value="Professional", label="Tone")
        hashtags = gr.Textbox(label="Hashtags (comma separated)", placeholder="ai, generativeai, datascience")

    with gr.Row():
        use_llm = gr.Checkbox(value=False, label="Use LLM Copywriter")
        li_max_words = gr.Slider(100, 350, value=180, step=10, label="LinkedIn max words")
        tweet_max_chars = gr.Slider(120, 280, value=270, step=5, label="X/Tweet max chars")
        article_max_words = gr.Slider(400, 1200, value=800, step=50, label="Article/Medium max words")

    run_btn = gr.Button("Generate")

    gr.Markdown("### Strategy Output")
    strategy_md = gr.Markdown()

    gr.Markdown("### Platform Copy")
    social_md = gr.Markdown()

    download_file = gr.File(label="Download", interactive=False)

    run_btn.click(
        fn=generate,
        inputs=[brand, audience, objective, platform, tone, hashtags, use_llm,
                li_max_words, tweet_max_chars, article_max_words],
        outputs=[strategy_md, social_md, download_file]
    )

# In Colab, share=True is handy for preview links
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1109c5b0956bd957e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Adding logo , layout and more customized gradio UI

In [ ]:
from crewai import Agent, Task, Crew
import gradio as gr
import re
from datetime import datetime
from pathlib import Path

# --- Logo: convert GitHub page URL to raw if needed ---
def to_raw_github(url: str) -> str:
    # Accepts both blob and raw URLs; converts blob → raw
    return url.replace("https://github.com/", "https://raw.githubusercontent.com/").replace("/blob/", "/")

LOGO_URL = to_raw_github("https://github.com/Decoding-Data-Science/airesidency/blob/main/dds_logo.jpg")

# ----------------------------
# Your existing agents (unchanged)
# ----------------------------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,
    verbose=True,
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,
)

# Optional: focused social copywriter (only if toggled)
social_copywriter = Agent(
    role="Social Copywriter",
    goal="Turn strategy into platform-appropriate copy with strong hooks and clear CTAs.",
    backstory=("Skilled at LinkedIn thought-leadership, X brevity, and long-form posts that convert."),
    allow_delegation=False,
    verbose=False,
)

# ----------------------------
# Core crew (unchanged)
# ----------------------------
def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        verbose=True
    )
    return crew.kickoff()

# ----------------------------
# Helpers
# ----------------------------
def _first_n_points(text: str, n: int = 5):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    bullets = []
    for l in lines:
        if l.startswith(("-", "*", "•")) or re.match(r"^\d+[\.\)]", l) or len(l) > 50:
            bullets.append(l.lstrip("-*• ").strip())
        if len(bullets) >= n: break
    if not bullets:
        parts = [p.strip() for p in text.split("\n\n") if p.strip()]
        bullets = parts[:n]
    return bullets[:n]

def _hashtags(csv_tags: str) -> str:
    tags = [t.strip().replace("#", "") for t in (csv_tags or "").split(",") if t.strip()]
    return "" if not tags else " " + " ".join(f"#{t}" for t in tags)

def _truncate_chars(s: str, max_chars: int) -> str:
    return s if len(s) <= max_chars else s[:max_chars - 1] + "…"

# ----------------------------
# Lightweight templates (no extra LLM call)
# ----------------------------
def tpl_linkedin(strategy, brand, audience, objective, hashtags, max_words=180):
    hook = f"{brand}: a sharper path to {objective} for {audience}."
    pts = "\n".join([f"- {p}" for p in _first_n_points(strategy, 5)])
    body = f"""**{hook}**

**Key insights**
{pts}

**Next 90 days**
- Prove the positioning with fast tests
- Double down on channels with strongest signals
- Track leading KPIs weekly

CTA: Comment “PLAYBOOK” if you want the GTM outline.{_hashtags(hashtags)}
"""
    words = body.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else body

def tpl_tweet(strategy, brand, audience, objective, hashtags, max_chars=270):
    points = _first_n_points(strategy, 3)
    msg = f"{brand} → {objective} for {audience}: " + " | ".join(_truncate_chars(p, 80) for p in points)
    return _truncate_chars(msg + _hashtags(hashtags), max_chars)

def tpl_article(strategy, brand, audience, objective, hashtags, max_words=800):
    intro = (f"{brand} is targeting {audience} to achieve {objective}. "
             "Here’s the distilled market analysis, GTM strategy, and a 90-day plan.")
    text = f"""# {brand}: GTM Playbook for {audience}

## Why this matters
{intro}

## Strategy in brief
- Positioning & messaging pillars
- Priority segments & channel mix
- Offers/CTAs, KPI tree
- 90-day roadmap & budget split

## Research & Strategy Notes
{strategy}

## What to do next
1) Validate 1–2 offers with tight ICP cohorts
2) Launch a 2-channel test with weekly KPI reviews
3) Scale what converts, archive what doesn’t

*Updated: {datetime.utcnow().strftime("%Y-%m-%d")}*{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

# ----------------------------
# Optional LLM copywriter (reuses Social Copywriter agent)
# ----------------------------
def llm_copywriter(strategy_text, brand, audience, objective, tone, platform,
                   hashtags, li_words, tweet_chars, article_words):
    limits = {
        "LinkedIn": f"≈ {li_words} words",
        "X (Twitter)": f"≤ {tweet_chars} chars",
        "Article": f"≈ {article_words} words",
    }
    req = f"""Create a {platform} post from the GTM strategy.

Brand: {brand}
Audience: {audience}
Objective: {objective}
Tone: {tone}
Hashtags: {hashtags or '(none)'}
Length limit: {limits.get(platform, 'keep concise')}

Requirements:
- Strong first-line hook
- 1–3 concrete insights from the strategy (no clichés)
- Clear CTA
- Respect platform style and length

--- STRATEGY ---
{strategy_text}
"""
    task = Task(description=req, expected_output=f"{platform} copy ready to publish.", agent=social_copywriter)
    crew = Crew(agents=[social_copywriter], tasks=[task], verbose=False)
    return crew.kickoff()

# ----------------------------
# Generation wrapper
# ----------------------------
PLATFORMS = ["LinkedIn", "X (Twitter)", "Article"]  # Reduced for better UX

def generate(product_brand, target_audience, objective,
             platform, tone, hashtags, use_llm,
             li_max_words, tweet_max_chars, article_max_words):

    if not product_brand or not target_audience or not objective:
        return "Please fill Brand, Audience, and Objective.", "", None

    # 1) Run the main Crew once
    strategy = run_marketing_crew(product_brand, target_audience, objective)

    # 2) Copy route
    if use_llm:
        social = llm_copywriter(strategy, product_brand, target_audience, objective,
                                tone, platform, hashtags,
                                li_max_words, tweet_max_chars, article_max_words)
    else:
        if platform == "LinkedIn":
            social = tpl_linkedin(strategy, product_brand, target_audience, objective, hashtags, li_max_words)
            fname = f"linkedin_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "X (Twitter)":
            social = tpl_tweet(strategy, product_brand, target_audience, objective, hashtags, tweet_max_chars)
            fname = f"tweet_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        else:  # Article
            social = tpl_article(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"article_{re.sub(r'\\W+','_',product_brand.lower())}.md"

        out_path = Path(fname).resolve()
        out_path.write_text(social, encoding="utf-8")
        return strategy, social, str(out_path)

    # Save LLM result with reasonable extension
    name_map = {"LinkedIn": "linkedin", "X (Twitter)": "tweet", "Article": "article"}
    fname = f"{name_map.get(platform,'post')}_{re.sub(r'\\W+','_',product_brand.lower())}.md"
    out_path = Path(fname).resolve()
    out_path.write_text(social, encoding="utf-8")
    return strategy, social, str(out_path)

# ----------------------------
# Theming & Layout (2 columns with header)
# ----------------------------
theme = gr.themes.Soft(primary_hue="indigo", neutral_hue="slate")

CUSTOM_CSS = """
#header {display:flex; align-items:center; gap:16px; padding:10px 14px; border-radius:12px;
         background: linear-gradient(90deg, #eef2ff, #f8fafc); border:1px solid #e5e7eb;}
#header img {width:44px; height:44px; object-fit:contain; border-radius:8px;}
#header .title {font-weight:700; font-size:18px; color:#111827;}
#header .subtitle {font-size:13px; color:#6b7280;}
.card {border:1px solid #e5e7eb; border-radius:12px; padding:12px; background:#ffffff;}
"""

with gr.Blocks(title="DDS Marketing Crew → Social Content", theme=theme, css=CUSTOM_CSS) as demo:
    with gr.Row():
        with gr.Column(scale=12):
            with gr.Row(elem_id="header"):
                gr.Image(value=LOGO_URL, show_label=False, interactive=False, height=48, width=48)
                gr.HTML("""
                    <div>
                      <div class="title">Decoding Data Science — Marketing Strategy → Social Generator</div>
                      <div class="subtitle">Run the Analyst → Strategist → Creator pipeline, then produce a platform-ready post.</div>
                    </div>
                """)

    with gr.Row():
        # Left: Inputs
        with gr.Column(scale=5):
            with gr.Group(elem_classes="card"):
                brand = gr.Textbox(label="Product/Brand", placeholder="e.g., DDS AI Residency", autofocus=True)
                audience = gr.Textbox(label="Target Audience", placeholder="e.g., Data science beginners in MENA")
                objective = gr.Textbox(label="Objective", placeholder="e.g., Drive applications for Cohort 8")

            with gr.Group(elem_classes="card"):
                platform = gr.Dropdown(choices=PLATFORMS, value="LinkedIn", label="Platform")
                tone = gr.Dropdown(choices=["Professional", "Friendly", "Bold", "Educational", "Conversational"],
                                   value="Professional", label="Tone")
                hashtags = gr.Textbox(label="Hashtags (comma separated)", placeholder="ai, generativeai, datascience")
                use_llm = gr.Checkbox(value=False, label="Use LLM Copywriter (higher fidelity)")

            with gr.Group(elem_classes="card"):
                li_max_words = gr.Slider(100, 350, value=180, step=10, label="LinkedIn max words")
                tweet_max_chars = gr.Slider(120, 280, value=270, step=5, label="X/Tweet max chars")
                article_max_words = gr.Slider(400, 1200, value=800, step=50, label="Article max words")

            run_btn = gr.Button("Generate Strategy → Post", variant="primary")

        # Right: Outputs
        with gr.Column(scale=7):
            with gr.Accordion("Strategy Output (from Crew)", open=True):
                strategy_md = gr.Markdown(value="*(Will appear here after generation)*")

            with gr.Group(elem_classes="card"):
                gr.Markdown("**Platform Copy** (editable; use the copy icon)")
                social_tb = gr.Textbox(lines=18, show_copy_button=True, label=None)

            download_file = gr.File(label="Download", interactive=False)

    run_btn.click(
        fn=generate,
        inputs=[brand, audience, objective, platform, tone, hashtags, use_llm,
                li_max_words, tweet_max_chars, article_max_words],
        outputs=[strategy_md, social_tb, download_file]
    )

# In Colab, share=True is handy
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54725b77fed75b850a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
